<h1 align=center style=color:green;><font size = 5>Segmenting and Clustering Neighborhoods in Toronto_Part 1</font></h1>
<h2 style=color:red>QUESTION 1<h2/>

In [3]:
#we start by importing the needed libraries
import requests
import numpy as np
import pandas as pd # library for data analsysis
from pandas import Series, DataFrame
from bs4 import BeautifulSoup




<h3 style=color:red>Scraping the list of postalcodes, Boroughs and Neighborhod in Canada<h3/>

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
requests.get(url).text
file = BeautifulSoup(requests.get(url).text, 'xml')
Table = file.find('table')
#Table

<h3 style=color:red>Creating a data frame with column names only for storing the scraped data<h3/>

In [5]:
df = DataFrame(columns=['PostalCode','Borough','Neighborhood'])
df

,PostalCode,Borough,Neighborhood


<h3 style=color:red>Looping through the table to extract the row data and append to the created Dataframe<h3/>

In [6]:
for table_row in Table.find_all('tr'):
 row_inputs = []
 for row_data in table_row.find_all('td'):
     row_inputs.append(row_data.text.strip())
 if len(row_inputs)==3:
    df.loc[len(df)] = row_inputs
    
df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3 style=color:red>Removing rows where Borough is not assigned<h3/>

In [7]:
df = df[df['Borough']!='Not assigned']
df.head(5)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3 style=color:red>Grouping the Dataframe by postal codes to ensure the duplicate postal coode are assigned uniquely to their respectives neighborhood(combined into one)<h3/>

In [27]:
grup_df = df.groupby('PostalCode')['Neighborhood']
grup_df = grup_df.apply(lambda x: "%s" % ', '.join(x))
grup_df = grup_df.reset_index(drop=False)
grup_df.rename(columns = {'Neighborhood':'df_merged'},inplace=True)
#grup_df.head(2)

In [67]:
df_merged = pd.merge(df, grup_df, on= 'PostalCode')
df_merged.drop_duplicates(inplace=True)
#df_merged.shape
df_merged.drop(['Neighborhood'],axis=1, inplace=True)
df_merged.rename(columns={'grouped_Neighborhood':'Neighborhood'}, inplace=True)

df_merged.head()

,PostalCode,Borough,df_merged
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [68]:
df_merged.shape

(103, 3)

<h1>PART 1 ENDS HERE<h1/>






<h1 align=center style=color:green;><font size = 5>Segmenting and Clustering Neighborhoods in Toronto_Part 2</font></h1>
<h2 style=color:blue>PROBLEM 2<h2/>

    
    
    

<h3 a style=color:blue>We have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we proceed to get the latitude and the longitude coordinates of each neighborhood.<h3/>

In [10]:
##alternative method for getting the location co-oordinates
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [11]:
#note that the co-ordinates off the locations has already been provided and stored in the below file
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [12]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3 align=center style=color:blue>Merging the Two tables containing the lattiitude and longitude of the locations to the already existing created Data frame<h3/> 

In [34]:

geo_df.rename(columns= {'Postal Code':'PostalCode'}, inplace=True)
geo_df.head()
merge_df = pd.merge(df_merged, geo_df,  on= 'PostalCode')
merge_df.rename(columns= {'df_merged':'Neighborhood'}, inplace=True)
merge_df.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [35]:
merge_df.shape

(103, 5)

<h1>part 2 ends here</h1>



<h1 align=center style=color:green;><font size = 5>Segmenting and Clustering Neighborhoods in Toronto_Part 3</font></h1>
<h2 style=color:purple>PROBLEM 3<h2/>

<h3 style="color:purple">Filtering to get only boroughs that contain the word Toronto</h3>


In [36]:
toronto_df=merge_df[merge_df['Borough'].str.contains("Toronto")]
toronto_df.head()
#toronto_df.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [20]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Solving environment: | ^C
failed

CondaError: KeyboardInterrupt

Libraries imported.


 <h3 style="color:purple">Use geopy library to get the latitude and longitude values of TORONTO</h3>


<h5 style="color:purple">In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>T_explorer</em>, as shown below.</h5>

In [22]:
address = 'Toronto, TOR'

geolocator = Nominatim(user_agent="T_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7627912, -79.4064452.


In [1]:
#installing/importing the folium library to plotting map
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

<h5 style="color:purple">create map of TORONTO using latitude and longitude values
</h5>

In [37]:


# create map of TORONTO using latitude and longitude values
map_TORONTO = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TORONTO)  
    
map_TORONTO


<h3 style="color:purple">Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.<br>

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods Central Toronto. So let's slice the original dataframe and create a new dataframe of the C_TOR_data.
</h3>

In [38]:
toronto_df.Borough
C_TOR_data = toronto_df[toronto_df['Borough'] == 'Central Toronto'].reset_index(drop=True)
C_TOR_data.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678


In [39]:
#lets the geographical coordinates of Central tooronto.
address = 'Central Toronto, TOR'

geolocator = Nominatim(user_agent="T_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.7627912, -79.4064452.



<h3 style="color:purple">As we did with all of Toronto, let's visualize Central toronto and  the neighborhoods in it.
</h3>

In [41]:
#MAP HERE
# create map of Central toronto using latitude and longitude values
map_TOR_Central = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(C_TOR_data['Latitude'], C_TOR_data['Longitude'], C_TOR_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TOR_Central)  
    
map_TOR_Central

In [72]:
CLIENT_ID = '2IW35JEHROS4UOWMJQJUNFE5Y4IWYIBERQSNIDKQSBNAHAC3' # your Foursquare ID
CLIENT_SECRET = 'SZBI4KNZETCPDD3BDHWX4P0ZIFKYX3KJQCLX0WM11HSURGMI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = '200'


print('LIMIT:' + LIMIT )
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT:200
Your credentails:
CLIENT_ID: 2IW35JEHROS4UOWMJQJUNFE5Y4IWYIBERQSNIDKQSBNAHAC3
CLIENT_SECRET:SZBI4KNZETCPDD3BDHWX4P0ZIFKYX3KJQCLX0WM11HSURGMI


CENTRAL TORONTO<h1 style = color:purple>Exploring venues in Central Toronto<h1/>

<h3 style="color:purple">now lets create a function that extracts nearby venues in Central toronto
</h3>

In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h5 style="color:purple">Now write the code to run the above function on each neighborhood and create a new dataframe called Central_tor_venues.
</h5>

In [54]:
# type your answer here

Central_tor_venues = getNearbyVenues(names=C_TOR_data['Neighborhood'],
                                   latitudes=C_TOR_data['Latitude'],
                                   longitudes=C_TOR_data['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


In [55]:
#Let's check the size of the resulting dataframe and the first five rows
print(Central_tor_venues.shape)
Central_tor_venues.head()

(117, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Dim Sum Deluxe,43.726953,-79.394260,Dim Sum Restaurant
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service


In [56]:
#Let's check how many venues were returned for each neighborhood
Central_tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,34,34,34,34,34,34
Davisville North,7,7,7,7,7,7
"Forest Hill North & West, Forest Hill Road Park",5,5,5,5,5,5
Lawrence Park,4,4,4,4,4,4
"Moore Park, Summerhill East",3,3,3,3,3,3
"North Toronto West, Lawrence Park",23,23,23,23,23,23
Roselawn,2,2,2,2,2,2
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",16,16,16,16,16,16
"The Annex, North Midtown, Yorkville",23,23,23,23,23,23


<h1 align=center style = color:purple;>ANALYZING EACH NEIGHBORHOOD IN CENTRAL TORONTO<h1/>

In [57]:
# one hot encoding
central_tor_onehot = pd.get_dummies(Central_tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_tor_onehot['Neighborhood'] = Central_tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [central_tor_onehot.columns[-1]] + list(central_tor_onehot.columns[:-1])
central_tor_onehot = central_tor_onehot[fixed_columns]

central_tor_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,...,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3 style="color:purple">Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
</h3>

In [58]:
CENT_TOR_grouped = central_tor_onehot.groupby('Neighborhood').mean().reset_index()
#TORONTO_grouped


In [59]:
#And let's examine the new dataframe size.
CENT_TOR_grouped.shape

(9, 70)

<h3 style="color:purple">Let's print each neighborhood along with the top 5 most common venues
</h3>

In [60]:
num_top_venues = 5

for hood in CENT_TOR_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = CENT_TOR_grouped[CENT_TOR_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0        Dessert Shop  0.09
1      Sandwich Place  0.09
2                 Gym  0.06
3  Italian Restaurant  0.06
4                Café  0.06


----Davisville North----
               venue  freq
0     Sandwich Place  0.14
1              Hotel  0.14
2  Food & Drink Shop  0.14
3   Department Store  0.14
4     Breakfast Spot  0.14


----Forest Hill North & West, Forest Hill Road Park----
              venue  freq
0          Bus Line   0.2
1             Trail   0.2
2     Jewelry Store   0.2
3  Sushi Restaurant   0.2
4              Park   0.2


----Lawrence Park----
                venue  freq
0  Dim Sum Restaurant  0.25
1         Swim School  0.25
2                Park  0.25
3            Bus Line  0.25
4  Light Rail Station  0.00


----Moore Park, Summerhill East----
                 venue  freq
0           Playground  0.33
1          Summer Camp  0.33
2           Restaurant  0.33
3  American Restaurant  0.00
4                 Park  0.00


----

<h3 style="color:purple">Let's put that into a pandas dataframe<br><br>
First, let's write a function to sort the venues in descending order.
</h3>

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h3 style="color:purple">Now let's create the new dataframe and display the top 10 venues for each neighborhood.
</h3>

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CENT_TOR_grouped['Neighborhood']

for ind in np.arange(CENT_TOR_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CENT_TOR_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Gym,Café,Indian Restaurant,Farmers Market
1,Davisville North,Gym,Food & Drink Shop,Department Store,Hotel,Sandwich Place,Breakfast Spot,Park,Farmers Market,Fast Food Restaurant,Flower Shop
2,"Forest Hill North & West, Forest Hill Road Park",Jewelry Store,Trail,Park,Sushi Restaurant,Bus Line,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop
3,Lawrence Park,Park,Swim School,Bus Line,Dim Sum Restaurant,Yoga Studio,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop
4,"Moore Park, Summerhill East",Summer Camp,Restaurant,Playground,Gourmet Shop,Gift Shop,Gas Station,Garden,Furniture / Home Store,Fried Chicken Joint,Dessert Shop


<h1 style="color:purple"> Clustering Neighborhoods in Central Toronto<h1/>
 <h3 style=color:purple;>Run k-means to cluster the neighborhood into 5 clusters.</h3>

In [63]:
kclusters = 5

C_TORONTO_grouped_clustering = CENT_TOR_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(C_TORONTO_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 4, 2, 0, 1, 0, 0], dtype=int32)

<h3 style="color:purple"> Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
</h3>

In [65]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Central_TOR_MERGED = C_TOR_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Central_TOR_MERGED = Central_TOR_MERGED.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Central_TOR_MERGED.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Dim Sum Restaurant,Yoga Studio,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Home Service,Garden,Fried Chicken Joint,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Furniture / Home Store
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym,Food & Drink Shop,Department Store,Hotel,Sandwich Place,Breakfast Spot,Park,Farmers Market,Fast Food Restaurant,Flower Shop
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,3,Jewelry Store,Trail,Park,Sushi Restaurant,Bus Line,Food & Drink Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Clothing Store,Coffee Shop,Chinese Restaurant,Health & Beauty Service,Furniture / Home Store,Fast Food Restaurant,Diner,Mexican Restaurant,Dessert Shop,Park


<h3 style="color:purple">Finally, let's visualize the resulting clusters
</h3>
		

In [66]:
#MAP HERE
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Central_TOR_MERGED['Latitude'], Central_TOR_MERGED['Longitude'], Central_TOR_MERGED['Neighborhood'], Central_TOR_MERGED['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1 style = color:purple;>EXAMINING THE FIRST 3 CLUSTERS<h1/>

<h1 style=color:red;>FIRST CLUSTER<h1/>

In [202]:
Central_TOR_MERGED.loc[Central_TOR_MERGED['Cluster Labels'] == 0, Central_TOR_MERGED.columns[[1] + list(range(5, Central_TOR_MERGED.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,0,Gym,Food & Drink Shop,Department Store,Hotel,Sandwich Place,Breakfast Spot,Park,Farmers Market,Fast Food Restaurant,Flower Shop
4,Central Toronto,0,Clothing Store,Coffee Shop,Chinese Restaurant,Health & Beauty Service,Furniture / Home Store,Fast Food Restaurant,Diner,Mexican Restaurant,Dessert Shop,Park
5,Central Toronto,0,Sandwich Place,Café,Coffee Shop,American Restaurant,History Museum,Indian Restaurant,Flower Shop,Donut Shop,Liquor Store,Middle Eastern Restaurant
6,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Gym,Café,Indian Restaurant,Farmers Market
8,Central Toronto,0,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store,Pizza Place,Restaurant,American Restaurant


<h1 style=color:red;>SECOND CLUSTER<h1/>

In [203]:
Central_TOR_MERGED.loc[Central_TOR_MERGED['Cluster Labels'] == 1, Central_TOR_MERGED.columns[[1] + list(range(5, Central_TOR_MERGED.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Home Service,Garden,Fried Chicken Joint,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Furniture / Home Store


<h1 style=color:red;>THIRD CLUSTER<h1/>

In [204]:
Central_TOR_MERGED.loc[Central_TOR_MERGED['Cluster Labels'] == 2, Central_TOR_MERGED.columns[[1] + list(range(5, Central_TOR_MERGED.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,2,Summer Camp,Restaurant,Playground,Gourmet Shop,Gift Shop,Gas Station,Garden,Furniture / Home Store,Fried Chicken Joint,Dessert Shop
